# CUT

In [ ]:
!cd CUT && bash ./datasets/download_cut_dataset.sh grumpifycat

In [ ]:
!cd CUT && python3 train.py --dataroot ./datasets/grumpifycat --name grumpycat_FastCUT --CUT_mode FastCUT --gpu_ids -1

In [ ]:
!python3 -m visdom.server
# http://localhost:8097

# If address is used when starting visdom
# !lsof -i tcp:8097
# !kill -9 <PID>

In [ ]:
--no-flip
--n_epochs N_EPOCHS   number of epochs with the initial learning rate (default: 200)
--n_epochs_decay N_EPOCHS_DECAY
                    number of epochs to linearly decay learning rate to zero (default: 200)

# EPE

In [1]:
is_colab = False

if is_colab:
    !git clone https://github.com/mseg-dataset/mseg-api.git
    !git clone https://github.com/mseg-dataset/mseg-semantic.git
    !git clone https://github.com/isl-org/PhotorealismEnhancement.git

    !pip3 install -e mseg-api/.
    !pip3 install -e mseg-semantic/.
    !pip3 install faiss-gpu
    !pip3 install lpips
    !pip3 install kornia
    !pip3 install kornia[x]
    !pip3 install pytorch torchvision cudatoolkit==10.2
    !pip3 install scikit-image
    !pip3 install imageio
    !pip3 install tqdm
    !pip3 install -e PhotorealismEnhancement/code/.

    from google.colab import drive
    drive.mount('/content/drive')

from tqdm import tqdm
import imageio
import numpy as np
import os
import pandas as pd
import warnings


game = 'halo2'
warnings.filterwarnings('ignore')

In [ ]:
# !convert -size 960x480 xc:black nan_gt_sseg.png
# !ffmpeg -ss 00:00:00 -to 00:38:44 -i halo2_cg_comparison_raw.mp4 -c copy halo2_cg_comparison.mp4
# !rm halo2_cg_comparison_raw.mp4
# !mkdir snaps
# !ffmpeg -i halo2_cg_comparison.mp4 -r 0.02 snaps/snap_%04d.png
# !convert snaps/snap_*.png -gravity East -crop 50%x100%+0+0 origs/orig_%04d.png
# !convert snaps/snap_*.png -gravity West -crop 50%x100%+0+0 remasters/rmst_%04d.png
# !mogrify +repage -crop 960x480+0+270 rmst_*.png

In [2]:
def filter_raw_snaps(folder):
    # Certain input images are all black / white so would raise error
    files = os.listdir(folder)
    valid_files = []

    for path in tqdm(files):
        full_path = os.path.join(folder, path)
        
        try:
            test = np.clip(imageio.imread(full_path).astype(np.float32) / 255.0, 0.0, 1.0)[:,:,:3]
            valid_files.append(full_path)
        except Exception:
            pass

    return valid_files

In [7]:
if is_colab:
    orig_folder = '/content/drive/Othercomputers/My MacBook Pro/GoogleDrive/origs'
    rmst_folder = '/content/drive/Othercomputers/My MacBook Pro/GoogleDrive/remasters'
else:
    orig_folder = f'data/{game}/origs'
    rmst_folder = f'data/{game}/remasters'
    
pd.Series(filter_raw_snaps(orig_folder)).to_csv(f'data/{game}/orig_files.csv', index=False, header=False)
pd.Series(filter_raw_snaps(rmst_folder)).to_csv(f'data/{game}/rmst_files.csv', index=False, header=False)

  0%|                                                | 0/61 [00:00<?, ?it/s]/var/folders/gc/y6z42t0n7v5bkllyb2j25wk40000gn/T/ipykernel_25570/3672624106.py:10: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  test = np.clip(imageio.imread(full_path).astype(np.float32) / 255.0, 0.0, 1.0)[:,:,:3]
100%|███████████████████████████████████████| 61/61 [00:01<00:00, 56.27it/s]


In [17]:
os.makedirs(f'data/{game}/gbuffers')
!python3 PhotorealismEnhancement/code/epe/dataset/generate_fake_gbuffers.py {game}_GBF data/{game}/orig_files.csv --out_dir data/{game}/gbuffers

NOTE: Redirects are currently not supported in Windows or MacOs.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Computing mean/std...
0it [00:00, ?it/s]NOTE: Redirects are currently not supported in Windows or MacOs.
58it [00:07,  7.27it/s]
100%|███████████████████████████████████████████| 58/58 [01:43<00:00,  1.78s/it]


In [21]:
!python3 PhotorealismEnhancement/code/epe/matching/feature_based/collect_crops.py {game}_Orig data/{game}/orig_files.csv --out_dir data/{game}
!python3 PhotorealismEnhancement/code/epe/matching/feature_based/collect_crops.py {game}_Rmst data/{game}/rmst_files.csv --out_dir data/{game}

NOTE: Redirects are currently not supported in Windows or MacOs.
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Computing mean/std...
0it [00:00, ?it/s]NOTE: Redirects are currently not supported in Windows or MacOs.
58it [00:08,  7.22it/s]
Sampling crops...
  0%|                                                    | 0/58 [00:00<?, ?it/s]NOTE: Redir

In [25]:
!python3 PhotorealismEnhancement/code/epe/matching/feature_based/find_knn.py \
    data/{game}/crop_{game}_Orig.npz data/{game}/crop_{game}_Rmst.npz data/{game}/knn_{game}.npz -k 10

Found 870 crops for source dataset.
Found 900 crops for target dataset.


In [26]:
!python3 PhotorealismEnhancement/code/epe/matching/filter.py \
    data/{game}/knn_{game}.npz data/{game}/crop_{game}_Orig.csv data/{game}/crop_{game}_Rmst.csv 1.0 data/{game}/matched_crops_Halo2.csv

870it [00:00, 569007.40it/s]
900it [00:00, 617717.82it/s]
100%|██████████████████████████████████| 4557/4557 [00:00<00:00, 2420342.32it/s]


In [34]:
# https://github.com/isl-org/PhotorealismEnhancement/issues/26
!python3 PhotorealismEnhancement/code/epe/matching/feature_based/sample_matches.py \
    data/{game}/orig_files.csv data/{game}/crop_{game}_Orig.csv data/{game}/rmst_files.csv \
    data/{game}/crop_{game}_Rmst.csv data/{game}/knn_{game}.npz

In [36]:
# !python3 PhotorealismEnhancement/code/epe/matching/compute_weights.py \
#     data/{game}/matched_crops_{game}.csv 960 1080 data/{game}/crop_weights_{game}.npz
!python3 PhotorealismEnhancement/code/epe/matching/compute_weights.py \
    data/{game}/matched_crops_{game}.csv 480 960 data/{game}/crop_weights_{game}.npz

Computing density...
100%|████████████████████████████████████| 4557/4557 [00:00<00:00, 53171.47it/s]
Computing individual weights...
100%|████████████████████████████████████| 4557/4557 [00:00<00:00, 64752.92it/s]


In [18]:
orig_train = pd.read_csv(f'data/{game}/orig_files.csv', header=None).sort_values(0)
orig_train[1] = orig_train[0].str.replace('.png', '.npz', regex=False)
orig_train[1] = orig_train[1].str.replace('origs', 'gbuffers', regex=False)
orig_train[2] = orig_train[0].str.replace('.png', '_overlaid_classes.jpg', regex=False)
orig_train[2] = orig_train[2].str.replace('origs', 'orig_rbseg', regex=False)
# orig_train[3] = 'nan_gt_sseg.png'
orig_train[3] = orig_train[2]
orig_train = orig_train[orig_train.apply(lambda row: all([os.path.exists(row[x]) for x in range(orig_train.shape[1])]), axis=1)]
orig_train = orig_train[[0, 2, 1, 3]]
orig_train.to_csv(f'data/{game}/orig_train.txt', index=False, header=False)

# mseg_fmt = 'python3 -u mseg-semantic/mseg_semantic/tool/universal_demo.py --config=mseg-semantic/mseg_semantic/config/test/default_config_360_ms.yaml model_name mseg-3m model_path mseg-3m-1080p.pth input_file {}'
mseg_fmt = 'python3 -u mseg-semantic/mseg_semantic/tool/universal_demo.py --config=mseg-semantic/mseg_semantic/config/test/default_config_360_ss.yaml model_name mseg-3m model_path mseg-3m-1080p.pth input_file {}'

for x in tqdm(orig_train[0].values):
    res = os.system(mseg_fmt.format(x))
    break
    if res:
        print(f'{x} failed with exit code of {res}')

# for x in os.listdir('.'):
#     if x.endswith('_gray.jpg'):
#         os.remove(x)

!mkdir data/{game}/orig_rbseg2 && mv *_overlaid_classes.jpg data/{game}/orig_rbseg2

  0%|                                            | 0/54 [00:00<?, ?it/s]/Users/milz/Desktop/Study/Columbia/CS4995_DeepVisionTopics/Project/mseg-semantic/mseg_semantic/model/seg_hrnet.py:579: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize_config_module(config_module="mseg_semantic.model"):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/functional.py:3734: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
  0%|                                            | 0/54 [00:09<?, ?it/s]

mkdir: data/halo2/orig_rbseg2: File exists


In [343]:
rmst_train = pd.read_csv(f'data/{game}/rmst_files.csv', header=None).sort_values(0)
rmst_train[1] = rmst_train[0].str.replace('.png', '_overlaid_classes.jpg', regex=False)
rmst_train[1] = rmst_train[1].str.replace('remasters', 'rmst_rbseg', regex=False)
rmst_train = rmst_train[rmst_train.apply(lambda row: all([os.path.exists(row[x]) for x in range(rmst_train.shape[1])]), axis=1)]
rmst_train.to_csv(f'data/{game}/rmst_train.txt', index=False, header=False)

for i, row in tqdm(rmst_train.iterrows()):
    if os.path.exists(row[1]):
        continue
        
    res = os.system(mseg_fmt.format(row[0]))
    
    if res:
        print(f'{row[0]} failed with exit code of {res}')

for x in os.listdir('.'):
    if x.endswith('_gray.jpg'):
        os.remove(x)

!mkdir data/{game}/rmst_rbseg && mv *_overlaid_classes.jpg data/{game}/rmst_rbseg

/Users/milz/Desktop/Study/Columbia/CS4995_DeepVisionTopics/Project/mseg-semantic/mseg_semantic/model/seg_hrnet.py:579: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize_config_module(config_module="mseg_semantic.model"):
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/functional.py:3734: UserWarning: nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.
  warnings.warn("nn.functional.upsample is deprecated. Use nn.functional.interpolate instead.")
/Users/milz/Desktop/Study/Columbia/CS4995_DeepVisionTopics/Project/mseg-semantic/mseg_semantic/model/seg_hrnet.py:579: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  with hydra.initialize_config_module(config_module="mseg_semantic.model"):
/Library/Fram

mkdir: data/halo2/rmst_rbseg: File exists


In [8]:
!python3 PhotorealismEnhancement/code/epe/EPEExperiment.py train data/{game}/train_halo2.yaml --log=info

NOTE: Redirects are currently not supported in Windows or MacOs.
Namespace(action='train', config=PosixPath('data/halo2/train_halo2.yaml'), log='info', log_dir=PosixPath('log'), gpu=0, no_safe_exit=False)
Traceback (most recent call last):
  File "/Users/milz/Desktop/Study/Columbia/CS4995_DeepVisionTopics/Project/PhotorealismEnhancement/code/epe/EPEExperiment.py", line 390, in <module>
    raise
RuntimeError: No active exception to reraise
